Last updated: 2025-Aug-22

Udaya Maurya (udaya_cbscients@yahoo.com, telegram: https://t.me/udy11)

Source: https://github.com/udy11, https://gitlab.com/udy11

## psiViz

This code computes poloidal flux (ψ) and poloidal magnetic field components (B_R, B_Z) due to poloidal coil currents in an axisymmetric Tokamak. The poloidal coil currents can be varied and changes in ψ, B_R and B_Z can be visualized live. Optionally, a D-shaped plasma can be added and its combined effect can also be observed. Please note that the ψ, B_R and B_Z computed from this code is not equivalent to the ones obtained from an MHD Equilibrium code, but it only provides a gross visualization of the expected profiles. All values are in SI units unless specifically mentioned (like input coil currents that are in Mega Ampere).

ALL YOU NEED TO DO:
1. Set Tokamak related parameters in "Main inputs"
2. Optionally, add plasma in the "Add plasma" cell
3. Run until the last cell

INPUTS:
- In *Main Inputs* cell
    1. nr, nz = Number of grid points in R (horizontal) and Z (vertical) directions, try to keep them low otherwise live plotting could be slow (int)
    2. r1, z1 = 1D arrays specifying grid in R and Z directions (1d numpy array)
    3. Rmaj = Major Radius of Tokamak (float)
    4. ccoils = List of poloidal field coils position, size, turns and name (list of list)
    5. Icoil_init = Initial guess and step to change coil currents (in MA), first entry shows which coils to treat together (list of list)
    6. drf, dzf = Filament sizes to break each coil into (float)
    7. rzlims = Limiter cooridnates, only used in plotting (2d numpy array)
    8. rzvesl = Vessel cooridnates, only used in plotting (2d numpy array)
- In *Add Plasma* cell (optional)
    1. R0, Z0 = center of current density (float)
    2. Ip = plasma current (float)
    3. Rmin = minor radius within which current density is confined (float)
    4. elongation = function specifying how elongation varies with minor radius (float function of 1 variable)
    5. triangularity = function specifying how triangularity varies with minor radius (float function of 1 variable)
    6. squareness = function specifying how squareness varies with minor radius (float function of 1 variable)

OUTPUTS:
- Interactive plot in the last cell

In [ ]:
import numpy as np
import scipy.special
import scipy.interpolate
import matplotlib.pyplot as plt
import matplotlib.patches
import matplotlib.path
import ipywidgets
from IPython.display import display
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
def greenGS(R, Z, Rp, Zp):
    kappa2 = 4.0*R*Rp / ((R+Rp)**2 + (Z-Zp)**2)
    return np.sqrt(R*Rp/kappa2) * ((1.0 - 0.5 * kappa2) * scipy.special.ellipk(kappa2) - scipy.special.ellipe(kappa2))

In [ ]:
# Main Inputs, try to keep nr, nz low
nr = 51    # number of grid points in R
nz = 81    # number of grid points in Z
r1 = np.linspace(0.5, 8.0, nr)    # R-grid, try not to have 0 in array
z1 = np.linspace(-5.5, 5.5, nz)    # Z-grid
Rmaj = 4.4    # Major Radius in meter
ccoils = [[ 1.09,  0.00, 0.325, 6.730, 1900, 'CS'],      # 0 ###
          [ 2.09,  4.93, 0.741, 0.741,  481, 'PF1T'],      # 1 ###
          [ 6.40,  4.65, 0.552, 0.552,  267, 'PF2T'],      # 2 ###
          [ 7.43,  2.80, 0.574, 0.574,  288, 'PF3T'],      # 3 ###
          [ 7.43, -2.80, 0.574, 0.574,  288, 'PF3B'],      # 4 ###
          #[ 6.40, -4.76, 0.560, 0.560,  274, 'PF2B'],      # 5 ###
          [ 6.40, -4.65, 0.552, 0.552,  267, 'PF2B'],      # 5 ###
          [ 2.09, -4.93, 0.741, 0.741,  481, 'PF1B']]      # 6 ###
Icoil_init = [[(0,), 50, 1],
               [(1, 6), -2, 1],
               [(2, 5), 5, 1],
               [(3, 4), -4, 1]]
drf, dzf = 0.05, 0.05
rzlims = np.array([(2.637, 1.2185), (2.637, 1.1188), (2.637, 1.0191), (2.637, 0.91934), (2.637, 0.81962), (2.637, 0.7199), (2.637, 0.62018), (2.637, 0.52046), (2.637, 0.42074), (2.637, 0.32102), (2.637, 0.2213), (2.637, 0.12158), (2.637, 2.19E-02), (2.637, -7.79E-02), (2.637, -0.17758), (2.637, -0.2773), (2.637, -0.37702), (2.637, -0.47674), (2.637, -0.57646), (2.637, -0.67618), (2.637, -0.7759), (2.637, -0.87562), (2.637, -0.97534), (2.637, -1.0751), (2.637, -1.1748), (2.637, -1.2745), (2.6379, -1.3742), (2.6438, -1.4737), (2.6554, -1.5728), (2.6726, -1.671), (2.6953, -1.7681), (2.7234, -1.8637), (2.7569, -1.9576), (2.7956, -2.0495), (2.8395, -2.139), (2.8884, -2.2259), (2.9421, -2.31), (3.0005, -2.3908), (3.0633, -2.4682), (3.1304, -2.5419), (3.2016, -2.6118), (3.2766, -2.6775), (3.3552, -2.7388), (3.4371, -2.7956), (3.5222, -2.8477), (3.61, -2.8949), (3.7003, -2.9371), (3.7929, -2.974), (3.8875, -3.0057), (3.9837, -3.032), (4.0812, -3.0528), (4.1797, -3.0681), (4.2789, -3.0778), (4.3785, -3.0819), (4.4782, -3.0803), (4.5777, -3.0731), (4.6766, -3.0603), (4.7746, -3.0419), (4.8714, -3.018), (4.9667, -2.9886), (5.0601, -2.954), (5.1515, -2.9141), (5.2405, -2.8691), (5.3268, -2.8191), (5.4101, -2.7644), (5.4902, -2.705), (5.5668, -2.6412), (5.6397, -2.5732), (5.7087, -2.5011), (5.7734, -2.4253), (5.8338, -2.346), (5.8895, -2.2633), (5.9406, -2.1776), (5.9867, -2.0892), (6.0277, -1.9984), (6.0635, -1.9053), (6.094, -1.8104), (6.1191, -1.71E+00), (6.1387, -1.6161), (6.1527, -1.5174), (6.1612, -1.4181), (6.164, -1.3184), (6.164, -1.2187), (6.164, -1.1189), (6.164, -1.0192), (6.164, -0.91949), (6.164, -0.81977), (6.164, -0.72005), (6.164, -0.62033), (6.164, -0.52061), (6.164, -0.42089), (6.164, -0.32117), (6.164, -0.22145), (6.164, -0.12173), (6.164, -2.20E-02), (6.164, 7.77E-02), (6.164, 0.17743), (6.164, 0.27715), (6.164, 0.37687), (6.164, 0.47659), (6.164, 0.57631), (6.164, 0.67603), (6.164, 0.77575), (6.164, 0.87547), (6.164, 0.97519), (6.164, 1.0749), (6.164, 1.1746), (6.164, 1.2744), (6.1631, 1.3741), (6.1572, 1.4736), (6.1456, 1.5726), (6.1284, 1.6708), (6.1058, 1.7679), (6.0776, 1.8636), (6.0442, 1.9575), (6.0054, 2.0494), (5.9615, 2.1389), (5.9127, 2.2258), (5.859, 2.3098), (5.8006, 2.3907), (5.7378, 2.4681), (5.6707, 2.5418), (5.5995, 2.6117), (5.5245, 2.6774), (5.4459, 2.7387), (5.364, 2.7955), (5.279, 2.8476), (5.1911, 2.8948), (5.1008, 2.937), (5.0082, 2.974), (4.9137, 3.0057), (4.8175, 3.032), (4.72, 3.0528), (4.6215, 3.0681), (4.5222, 3.0778), (4.4226, 3.0819), (4.3229, 3.0803), (4.2235, 3.0731), (4.1246, 3.0603), (4.0266, 3.0419), (3.9298, 3.018), (3.8345, 2.9887), (3.741, 2.954), (3.6496, 2.9142), (3.5606, 2.8692), (3.4743, 2.8192), (3.391, 2.7645), (3.3109, 2.7051), (3.2343, 2.6413), (3.1614, 2.5733), (3.0924, 2.5013), (3.0277, 2.4254), (2.9673, 2.3461), (2.9115, 2.2634), (2.8605, 2.1778), (2.8144, 2.0894), (2.7734, 1.9985), (2.7375, 1.9055), (2.707, 1.8105), (2.6819, 1.714), (2.6623, 1.6163), (2.6483, 1.5176), (2.6398, 1.4182), (2.637, 1.3185), (2.637, 1.2185)])
rzvesl = np.array([(2.319, 1.3185), (2.3214, 1.4182), (2.3286, 1.5177), (2.3405, 1.6167), (2.3571, 1.715), (2.3785, 1.8124), (2.4044, 1.9087), (2.435, 2.0036), (2.4701, 2.097), (2.5096, 2.1886), (2.5534, 2.2781), (2.6015, 2.3655), (2.6537, 2.4505), (2.7099, 2.5329), (2.7701, 2.6124), (2.8339, 2.6891), (2.9014, 2.7625), (2.9722, 2.8327), (3.0464, 2.8993), (3.1237, 2.9624), (3.2039, 3.0217), (3.2868, 3.077), (3.3724, 3.1283), (3.4602, 3.1755), (3.5503, 3.2184), (3.6422, 3.257), (3.736, 3.2911), (3.8312, 3.3206), (3.9278, 3.3456), (4.0254, 3.3659), (4.1239, 3.3815), (4.223, 3.3924), (4.3226, 3.3985), (4.4223, 3.3999), (4.522, 3.3965), (4.6214, 3.3882), (4.7202, 3.3753), (4.8184, 3.3576), (4.9156, 3.3353), (5.0116, 3.3083), (5.1062, 3.2767), (5.1992, 3.2407), (5.2903, 3.2002), (5.3795, 3.1554), (5.4663, 3.1064), (5.5507, 3.0533), (5.6325, 2.9962), (5.7115, 2.9353), (5.7874, 2.8706), (5.8601, 2.8024), (5.9296, 2.7308), (5.9954, 2.656), (6.0577, 2.578), (6.1161, 2.4972), (6.1706, 2.4137), (6.221, 2.3276), (6.2673, 2.2393), (6.3092, 2.1488), (6.3468, 2.0564), (6.3799, 1.9623), (6.4085, 1.8668), (6.4324, 1.77), (6.4517, 1.6721), (6.4663, 1.5735), (6.4762, 1.4742), (6.4812, 1.3746), (6.482, 1.2749), (6.482, 1.1752), (6.482, 1.0754), (6.482, 0.97569), (6.482, 0.87595), (6.482, 0.77621), (6.482, 0.67647), (6.482, 0.57673), (6.482, 0.47699), (6.482, 0.37725), (6.482, 0.27751), (6.482, 0.17777), (6.482, 7.80E-02), (6.482, -2.17E-02), (6.482, -0.12145), (6.482, -0.22119), (6.482, -0.32093), (6.482, -0.42067), (6.482, -0.52041), (6.482, -0.62015), (6.482, -0.71989), (6.482, -0.81963), (6.482, -0.91937), (6.482, -1.0191), (6.482, -1.1188), (6.482, -1.2186), (6.482, -1.3183), (6.4796, -1.4181), (6.4725, -1.5176), (6.4605, -1.6166), (6.4439, -1.7149), (6.4226, -1.8123), (6.3966, -1.9086), (6.366, -2.0035), (6.331, -2.0969), (6.2915, -2.1885), (6.2476, -2.2781), (6.1996, -2.3654), (6.1473, -2.4504), (6.0911, -2.5328), (6.031, -2.6124), (5.9672, -2.689), (5.8997, -2.7624), (5.8288, -2.8326), (5.7547, -2.8993), (5.6774, -2.9623), (5.5972, -3.0216), (5.5142, -3.077), (5.4287, -3.1283), (5.3409, -3.1755), (5.2508, -3.2184), (5.1589, -3.2569), (5.0651, -3.291), (4.9699, -3.3206), (4.8734, -3.3456), (4.7757, -3.3659), (4.6772, -3.3815), (4.5781, -3.3924), (4.4785, -3.3985), (4.3788, -3.3999), (4.2791, -3.3965), (4.1798, -3.3883), (4.0809, -3.3753), (3.9827, -3.3576), (3.8855, -3.3353), (3.7895, -3.3083), (3.6949, -3.2768), (3.6019, -3.2407), (3.5108, -3.2003), (3.4216, -3.1555), (3.3348, -3.1065), (3.2504, -3.0534), (3.1686, -2.9963), (3.0896, -2.9354), (3.0137, -2.8707), (2.9409, -2.8025), (2.8715, -2.7309), (2.8056, -2.656), (2.7434, -2.5781), (2.685, -2.4973), (2.6305, -2.4138), (2.58, -2.3277), (2.5338, -2.2394), (2.4918, -2.1489), (2.4542, -2.0565), (2.4211, -1.9624), (2.3925, -1.8669), (2.3686, -1.7701), (2.3493, -1.6722), (2.3347, -1.5736), (2.3248, -1.4744), (2.3198, -1.3748), (2.319, -1.275), (2.319, -1.1753), (2.319, -1.0755), (2.319, -0.9758), (2.319, -0.87606), (2.319, -0.77632), (2.319, -0.67658), (2.319, -0.57684), (2.319, -0.4771), (2.319, -0.37736), (2.319, -0.27762), (2.319, -0.17788), (2.319, -7.81E-02), (2.319, 2.16E-02), (2.319, 0.12134), (2.319, 0.22108), (2.319, 0.32082), (2.319, 0.42056), (2.319, 0.5203), (2.319, 0.62004), (2.319, 0.71978), (2.319, 0.81952), (2.319, 0.91926), (2.319, 1.019), (2.319, 1.1187), (2.319, 1.2185), (2.319, 1.3185)])

# Some Pre-processing
dr, dz = (r1[1] - r1[0], z1[1] - z1[0])    # spacings in R-Z grid
r2, z2 = np.meshgrid(r1, z1, indexing='xy')
mask_lim = matplotlib.path.Path(rzlims).contains_points(np.column_stack((r2.ravel(), z2.ravel()))).reshape(np.shape(r2))    # 2d array same size as r2, acts as mask with value 1 for points inside limiter, 0 for outside
J0 = np.zeros_like(r2)
psiJ = np.zeros_like(r2)
BRJ = np.zeros_like(r2)
BZJ = np.zeros_like(r2)

In [ ]:
# Add Plasma, skip if not needed
R0, Z0 = Rmaj, 0.0
Ip = 10.0e6
Rmin = 1.5 # 1.7635
elongation = lambda r: 1.0 + r * (1.7 - 1.0) / Rmin
triangularity = lambda r: 0.0 + r * (-0.3 - 0.0) / Rmin
squareness = lambda r: 0.0 + r * (0.05 - 0.0) / Rmin

# Computation of current density
th = np.linspace(0.0, 2.0 * np.pi, 129)
rm = np.linspace(0.0, Rmin, 129)
th2, rm2 = np.meshgrid(th, rm)
rJ = R0 + rm2 * np.cos(th2 + np.arcsin(triangularity(rm2) * np.sin(th2)))
zJ = Z0 + elongation(rm2) * Rmin * np.sin(th + squareness(rm2) * np.sin(2.0 * th))
J0 = (1-rm2**2/Rmin**2)
J0 = scipy.interpolate.griddata((rJ.flatten(), zJ.flatten()), J0.flatten(), (r2, z2), method='cubic', fill_value=0.0)
J0 = J0 * Ip / (np.sum(J0) * dr * dz)

# Computation of psi, BR and BZ due to J0 comuted above
psiJ = np.zeros_like(r2)
for jr in range(nr):
    for jz in range(nz):
        if J0[jz, jr] > 0.0:
            for ir in range(nr):
                for iz in range(nz):
                    if (ir != jr) and (iz != jz):
                        psiJ[iz, ir] += greenGS(r1[ir], z1[iz], r1[jr], z1[jz]) * J0[jz, jr]
psiJ *= 4.0e-7 * dr * dz
BRJ = -np.gradient(psiJ, dz, axis=0)/r2
BZJ = np.gradient(psiJ, dr, axis=1)/r2
del th, rm, th2, rm2, rJ, zJ, J0

In [ ]:
# To see the plots of J0, psiJ, BRJ, BZJ. Restart kernel if you've run "%matplotlib widget" cell below and these plots don't appear
plt.close()
cnts = []
ttls = (r'$J\,(A/m^2)$', r'$\psi\,(T\cdot m^2)$', r'$B_R\,(T)$', r'$B_Z\,(T)$')
fig, axs = plt.subplots(1, 4, figsize=(14, 3))
cnts.append(axs[0].contour(r2, z2, J0, levels=32))
cnts.append(axs[1].contour(r2, z2, psiJ, levels=32))
cnts.append(axs[2].contour(r2, z2, BRJ, levels=32))
cnts.append(axs[3].contour(r2, z2, BZJ, levels=32))
for k in range(4):
    axs[k].plot(rzlims[:,0], rzlims[:,1], linewidth=1, color='gray')
    axs[k].plot(rzvesl[:,0], rzvesl[:,1], linewidth=0.5, color='black')
    axs[k].set_title(ttls[k])
    axs[k].set_aspect(1)
    axs[k].grid()
    fig.colorbar(cnts[k], ax=axs[k])
plt.show()
plt.close()

In [ ]:
# Get psi, BR, BZ due to unit coil currents. Rerun this only if you changed grid, coil positions, coil sizes, filament numbers
upsis = []
uBRs = []
uBZs = []
for c in Icoil_range:
    upsi0 = 0.0
    for i in c[0]:
        upsi = np.zeros_like(r2)
        nrf = max(2, round(ccoils[i][2]/drf))
        nzf = max(2, round(ccoils[i][3]/dzf))
        rc = np.linspace(ccoils[i][0]-0.5*ccoils[i][2], ccoils[i][0]+0.5*ccoils[i][2], nrf)
        zc = np.linspace(ccoils[i][1]-0.5*ccoils[i][3], ccoils[i][1]+0.5*ccoils[i][3], nzf)
        for ir in range(nr):
            for iz in range(nz):
                for r in rc:
                    for z in zc:
                        upsi[iz, ir] += greenGS(r1[ir], z1[iz], r, z)
                upsi[iz, ir] *= 4.0e-7 / (nrf * nzf)
        upsi0 += upsi
        print(f'Done coil {i}')
    upsis.append(upsi0)
    uBRs.append(-np.gradient(upsi0, dz, axis=0)/r2)
    uBZs.append(np.gradient(upsi0, dr, axis=1)/r2)

In [ ]:
%matplotlib widget
ccs = [ipywidgets.FloatText(value=c[1], step=c[2], description=' + '.join([ccoils[i][5] for i in c[0]]), disabled=False) for c in Icoil_range]
ccs.append(ipywidgets.Checkbox(value=True, description='Plasma', disabled=False, indent=False))
box = ipywidgets.VBox(ccs)
display(box)

psi = 0.0
BR = 0.0
BZ = 0.0
for k in range(len(Icoil_range)):
    psi += upsis[k] * ccs[k].value * 1.0e6
    BR += uBRs[k] * ccs[k].value * 1.0e6
    BZ += uBZs[k] * ccs[k].value * 1.0e6

ttls = (r'$\psi\,(T\cdot m^2)$', r'$B_R\,(T)$', r'$B_Z\,(T)$')
plt.close()
fig, axs = plt.subplots(1, 3, figsize=(12, 4))
cnts = []
cbars = []
dividers = []
caxs = []
cnts.append(axs[0].contour(r2, z2, psi, linewidths=1, levels=32))
cnts.append(axs[1].contour(r2, z2, BR, linewidths=1, levels=np.linspace(np.min(BR*mask_lim), np.max(BR*mask_lim), 32)))
cnts.append(axs[2].contour(r2, z2, BZ, linewidths=1, levels=np.linspace(np.min(BZ*mask_lim), np.max(BZ*mask_lim), 32)))
for k in range(3):
    dividers.append(make_axes_locatable(axs[k]))
    caxs.append(dividers[k].append_axes('right', size='5%', pad=0.05))
    cbars.append(fig.colorbar(cnts[k], cax=caxs[k]))
    axs[k].set_aspect(1)
    axs[k].set_xlabel(r'$R\,(m)$')
    axs[k].set_title(ttls[k])
    for c in ccoils:
        axs[k].add_patch(matplotlib.patches.Rectangle((c[0]-0.5*c[2], c[1]-0.5*c[3]), c[2], c[3], facecolor=(1,1,1,0), edgecolor='black', linewidth=1))
    axs[k].plot(rzlims[:,0], rzlims[:,1], linewidth=1, color='gray')
    axs[k].plot(rzvesl[:,0], rzvesl[:,1], linewidth=0.5, color='black')
    axs[k].grid()
    axs[k].set_axisbelow(True)
axs[0].set_ylabel(r'$Z\,(m)$')
plt.tight_layout()
def update(change=None):
    global cnts, cbars
    if ccs[-1].value:
        psi = np.copy(psiJ)
        BR = np.copy(BRJ)
        BZ = np.copy(BZJ)
    else:
        psi = 0.0
        BR = 0.0
        BZ = 0.0
    for k in range(len(Icoil_range)):
        psi += upsis[k] * ccs[k].value * 1.0e6
        BR += uBRs[k] * ccs[k].value * 1.0e6
        BZ += uBZs[k] * ccs[k].value * 1.0e6
    for k in range(3):
        cnts[k].remove()
    cnts[0] = axs[0].contour(r2, z2, psi, linewidths=1, levels=32)
    cnts[1] = axs[1].contour(r2, z2, BR, linewidths=1, levels=np.linspace(np.min(BR*mask_lim), np.max(BR*mask_lim), 32))
    cnts[2] = axs[2].contour(r2, z2, BZ, linewidths=1, levels=np.linspace(np.min(BZ*mask_lim), np.max(BZ*mask_lim), 32))
    for k in range(3):
        caxs[k].cla()
        cbars[k] = fig.colorbar(cnts[k], cax=caxs[k])
    fig.canvas.draw_idle()

for c in ccs:
    c.observe(update, names='value')

update()